In [ ]:
!pip install opencv-python

# Ekstraksi frame dari video

In [ ]:
import cv2
import os
from google.colab import drive

# Load video Hubungkan Google Colab dengan Google Drive
drive.mount('/content/drive')
video_path = '/content/drive/MyDrive/sampel_video/orang_ramai_5.mp4'
output_folder = "frames"

# Buat folder untuk menyimpan frame
os.makedirs(output_folder, exist_ok=True)

# Membaca video
cap = cv2.VideoCapture(video_path)
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_path = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_path, frame)
    frame_count += 1

cap.release()
print(f"Frames saved to {output_folder}")


In [ ]:
import cv2
import os

# Load video Hubungkan Google Colab dengan Google Drive
drive.mount('/content/drive')
video_path = '/content/drive/MyDrive/sampel_video/penculikan.mp4'
output_folder = "frames"

# Buat folder untuk menyimpan frame
os.makedirs(output_folder, exist_ok=True)

# Membaca video
cap = cv2.VideoCapture(video_path)
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_path = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_path, frame)
    frame_count += 1

cap.release()
print(f"Frames saved to {output_folder}")


# Pelacakan objek menggunakan Optical Flow

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Gunakan path lengkap ke video
video_path = '/content/drive/MyDrive/sampel_video/orang_ramai_5.mp4'
cap = cv2.VideoCapture(video_path)

# Periksa apakah video berhasil dibuka
if not cap.isOpened():
    print("Error: Video tidak dapat dibuka. Periksa kembali path file.")
    exit()

# Parameter Optical Flow
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Ambil frame pertama
ret, old_frame = cap.read()
if not ret:
    print("Error: Tidak dapat membaca frame pertama dari video.")
    cap.release()
    exit()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, maxCorners=100,
                             qualityLevel=0.3, minDistance=7, blockSize=7)
if p0 is None:
    print("Tidak ada fitur yang dapat dilacak pada frame pertama.")
    cap.release()
    exit()

# Mask untuk menggambar lintasan
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Gunakan GaussianBlur untuk mengurangi noise
    frame_gray = cv2.GaussianBlur(frame_gray, (5, 5), 0)

    # Hitung Optical Flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Validasi Optical Flow
    if p1 is None or st is None:
        print("Gagal menghitung optical flow pada frame ini.")
        continue

    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Deteksi ulang fitur jika terlalu sedikit
    if len(good_new) < 5:
        print("Fitur terlalu sedikit, mendeteksi ulang...")
        p0 = cv2.goodFeaturesToTrack(frame_gray, mask=None, maxCorners=100,
                                     qualityLevel=0.3, minDistance=7, blockSize=7)
        if p0 is None:
            print("Tidak ada fitur yang ditemukan.")
            break
        mask = np.zeros_like(frame)
        continue

    # Gambar lintasan pergerakan fitur
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 0, 255), -1)

    img = cv2.add(frame, mask)
    cv2_imshow(img)

    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

    # Break jika ada input tombol (biasanya pada local, di Colab tidak berfungsi sama)
    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load video
cap = cv2.VideoCapture("/content/drive/MyDrive/sampel_video/penculikan.mp4")

# Parameter Optical Flow
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Ambil frame pertama
ret, old_frame = cap.read()
if not ret:
    print("Error: Tidak dapat membaca video.")
    cap.release()
    exit()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
if p0 is None:
    print("Tidak ada fitur yang dapat dilacak pada frame pertama.")
    cap.release()
    exit()

# Mask untuk menggambar
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Gunakan GaussianBlur untuk mengurangi noise
    frame_gray = cv2.GaussianBlur(frame_gray, (5, 5), 0)

    # Hitung Optical Flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Validasi Optical Flow
    if p1 is None or st is None:
        print("Gagal menghitung optical flow pada frame ini.")
        continue

    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Deteksi ulang fitur jika terlalu sedikit
    if len(good_new) < 5:
        print("Fitur terlalu sedikit, mendeteksi ulang...")
        p0 = cv2.goodFeaturesToTrack(frame_gray, mask=None, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
        if p0 is None:
            print("Tidak ada fitur yang ditemukan.")
            break
        mask = np.zeros_like(frame)
        continue

    # Gambar lintasan
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 0, 255), -1)

    img = cv2.add(frame, mask)
    cv2_imshow(img)

    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

    # Break jika ada input tombol
    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


# Klasifikasi untuk deteksi aktivitas

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import cv2
import os
from scipy.stats import describe
from google.colab import drive

# Mount Google Drive jika belum
try:
    drive.mount('/content/drive')
except:
    print("Google Drive sudah dimount atau terjadi error mounting")

def extract_motion_features(optical_flow_data):
    """
    Mengekstrak fitur dari data optical flow untuk klasifikasi.
    """
    try:
        # Hitung magnitude dari vektor gerakan
        magnitudes = np.sqrt(np.sum(optical_flow_data**2, axis=2))

        # Ekstrak statistik deskriptif
        stats = describe(magnitudes.flatten())

        return np.array([
            stats.mean,
            stats.variance,
            stats.skewness,
            stats.kurtosis,
            np.max(magnitudes),
            np.percentile(magnitudes, 75)
        ])
    except Exception as e:
        print(f"Error dalam ekstraksi fitur: {str(e)}")
        return None

def process_video(video_path):
    """
    Memproses video untuk mengekstrak fitur gerakan.
    """
    if not os.path.exists(video_path):
        print(f"Error: File video tidak ditemukan di {video_path}")
        return None

    try:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Tidak dapat membuka video di {video_path}")
            return None

        features_list = []
        ret, old_frame = cap.read()
        if not ret:
            print("Error: Tidak dapat membaca frame pertama")
            cap.release()
            return None

        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
        frame_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Hitung optical flow
            flow = cv2.calcOpticalFlowFarneback(
                old_gray, frame_gray,
                None, 0.5, 3, 15, 3, 5, 1.2, 0
            )

            # Ekstrak fitur
            features = extract_motion_features(flow)
            if features is not None:
                features_list.append(features)

            old_gray = frame_gray
            frame_count += 1

            # Print progress setiap 100 frame
            if frame_count % 100 == 0:
                print(f"Processed {frame_count} frames")

        cap.release()

        if len(features_list) == 0:
            print("Warning: Tidak ada fitur yang berhasil diekstrak")
            return None

        return np.array(features_list)

    except Exception as e:
        print(f"Error dalam pemrosesan video: {str(e)}")
        return None

def prepare_dataset():
    """
    Menyiapkan dataset untuk klasifikasi.
    """
    print("Memulai persiapan dataset...")

    # Path video
    normal_video_path = '/content/drive/MyDrive/sampel_video/orang_ramai_5.mp4'
    suspicious_video_path = '/content/drive/MyDrive/sampel_video/penculikan.mp4'

    # Validasi keberadaan file
    if not os.path.exists(normal_video_path):
        raise FileNotFoundError(f"Video normal tidak ditemukan di {normal_video_path}")
    if not os.path.exists(suspicious_video_path):
        raise FileNotFoundError(f"Video mencurigakan tidak ditemukan di {suspicious_video_path}")

    print("Memproses video normal...")
    normal_features = process_video(normal_video_path)
    if normal_features is None:
        raise ValueError("Gagal memproses video normal")

    print("Memproses video mencurigakan...")
    suspicious_features = process_video(suspicious_video_path)
    if suspicious_features is None:
        raise ValueError("Gagal memproses video mencurigakan")

    # Gabungkan fitur dan buat label
    X = np.vstack([normal_features, suspicious_features])
    y = np.hstack([
        np.zeros(len(normal_features)),
        np.ones(len(suspicious_features))
    ])

    print(f"Dataset siap: {len(X)} sampel total")
    return X, y

def train_classifier():
    """
    Melatih model klasifikasi.
    """
    try:
        print("Memulai pelatihan model...")
        X, y = prepare_dataset()

        # Bagi data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        # Normalisasi
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Latih model
        clf = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=42
        )
        clf.fit(X_train_scaled, y_train)

        # Evaluasi model
        train_score = clf.score(X_train_scaled, y_train)
        test_score = clf.score(X_test_scaled, y_test)
        print(f"Akurasi training: {train_score:.2%}")
        print(f"Akurasi testing: {test_score:.2%}")

        return clf, scaler

    except Exception as e:
        print(f"Error dalam pelatihan model: {str(e)}")
        return None, None

def analyze_video(video_path, clf, scaler):
    """
    Menganalisis video baru.
    """
    if clf is None or scaler is None:
        print("Error: Model klasifikasi atau scaler tidak tersedia")
        return None, None

    if not os.path.exists(video_path):
        print(f"Error: File video tidak ditemukan di {video_path}")
        return None, None

    try:
        features = process_video(video_path)
        if features is None:
            print("Error: Gagal mengekstrak fitur dari video")
            return None, None

        # Normalisasi fitur
        features_scaled = scaler.transform(features)

        # Prediksi
        predictions = clf.predict(features_scaled)
        suspicious_ratio = np.mean(predictions)

        return predictions, suspicious_ratio

    except Exception as e:
        print(f"Error dalam analisis video: {str(e)}")
        return None, None

# Jalankan proses training
print("Memulai proses training...")
clf, scaler = train_classifier()

if clf is not None and scaler is not None:
    # Analisis video
    test_video_path = '/content/drive/MyDrive/sampel_video/orang_ramai_6.mp4'
    print(f"\nMenganalisis video: {test_video_path}")

    result = analyze_video(test_video_path, clf, scaler)

    if result is not None:
        predictions, suspicious_ratio = result
        print(f"\nHasil Analisis:")
        print(f"Proporsi frame mencurigakan: {suspicious_ratio:.2%}")
        if suspicious_ratio > 0.5:
            print("Kesimpulan: Video mengandung aktivitas mencurigakan")
        else:
            print("Kesimpulan: Video menunjukkan aktivitas normal")
    else:
        print("Gagal menganalisis video")
else:
    print("Gagal melatih model")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Memulai proses training...
Memulai pelatihan model...
Memulai persiapan dataset...
Memproses video normal...
Processed 100 frames
Processed 200 frames
Memproses video mencurigakan...
Processed 100 frames
Processed 200 frames
Processed 300 frames
Processed 400 frames
Dataset siap: 625 sampel total
Akurasi training: 100.00%
Akurasi testing: 96.80%

Menganalisis video: /content/drive/MyDrive/sampel_video/orang_ramai_6.mp4
Processed 100 frames
Processed 200 frames
Processed 300 frames
Processed 400 frames

Hasil Analisis:
Proporsi frame mencurigakan: 41.28%
Kesimpulan: Video menunjukkan aktivitas normal


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Gunakan path lengkap ke video
video_path = '/content/drive/MyDrive/sampel_video/orang_ramai_6.mp4'
cap = cv2.VideoCapture(video_path)

# Periksa apakah video berhasil dibuka
if not cap.isOpened():
    print("Error: Video tidak dapat dibuka. Periksa kembali path file.")
    exit()

# Parameter Optical Flow
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Ambil frame pertama
ret, old_frame = cap.read()
if not ret:
    print("Error: Tidak dapat membaca frame pertama dari video.")
    cap.release()
    exit()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, maxCorners=100,
                             qualityLevel=0.3, minDistance=7, blockSize=7)
if p0 is None:
    print("Tidak ada fitur yang dapat dilacak pada frame pertama.")
    cap.release()
    exit()

# Mask untuk menggambar lintasan
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Gunakan GaussianBlur untuk mengurangi noise
    frame_gray = cv2.GaussianBlur(frame_gray, (5, 5), 0)

    # Hitung Optical Flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Validasi Optical Flow
    if p1 is None or st is None:
        print("Gagal menghitung optical flow pada frame ini.")
        continue

    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Deteksi ulang fitur jika terlalu sedikit
    if len(good_new) < 5:
        print("Fitur terlalu sedikit, mendeteksi ulang...")
        p0 = cv2.goodFeaturesToTrack(frame_gray, mask=None, maxCorners=100,
                                     qualityLevel=0.3, minDistance=7, blockSize=7)
        if p0 is None:
            print("Tidak ada fitur yang ditemukan.")
            break
        mask = np.zeros_like(frame)
        continue

    # Gambar lintasan pergerakan fitur
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 0, 255), -1)

    img = cv2.add(frame, mask)
    cv2_imshow(img)

    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

    # Break jika ada input tombol (biasanya pada local, di Colab tidak berfungsi sama)
    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import cv2
import os
from scipy.stats import describe
from google.colab import drive

# Mount Google Drive jika belum
try:
    drive.mount('/content/drive')
except:
    print("Google Drive sudah dimount atau terjadi error mounting")

def extract_motion_features(optical_flow_data):
    """
    Mengekstrak fitur dari data optical flow untuk klasifikasi.
    """
    try:
        # Hitung magnitude dari vektor gerakan
        magnitudes = np.sqrt(np.sum(optical_flow_data**2, axis=2))

        # Ekstrak statistik deskriptif
        stats = describe(magnitudes.flatten())

        return np.array([
            stats.mean,
            stats.variance,
            stats.skewness,
            stats.kurtosis,
            np.max(magnitudes),
            np.percentile(magnitudes, 75)
        ])
    except Exception as e:
        print(f"Error dalam ekstraksi fitur: {str(e)}")
        return None

def process_video(video_path):
    """
    Memproses video untuk mengekstrak fitur gerakan.
    """
    if not os.path.exists(video_path):
        print(f"Error: File video tidak ditemukan di {video_path}")
        return None

    try:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Tidak dapat membuka video di {video_path}")
            return None

        features_list = []
        ret, old_frame = cap.read()
        if not ret:
            print("Error: Tidak dapat membaca frame pertama")
            cap.release()
            return None

        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
        frame_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Hitung optical flow
            flow = cv2.calcOpticalFlowFarneback(
                old_gray, frame_gray,
                None, 0.5, 3, 15, 3, 5, 1.2, 0
            )

            # Ekstrak fitur
            features = extract_motion_features(flow)
            if features is not None:
                features_list.append(features)

            old_gray = frame_gray
            frame_count += 1

            # Print progress setiap 100 frame
            if frame_count % 100 == 0:
                print(f"Processed {frame_count} frames")

        cap.release()

        if len(features_list) == 0:
            print("Warning: Tidak ada fitur yang berhasil diekstrak")
            return None

        return np.array(features_list)

    except Exception as e:
        print(f"Error dalam pemrosesan video: {str(e)}")
        return None

def prepare_dataset():
    """
    Menyiapkan dataset untuk klasifikasi.
    """
    print("Memulai persiapan dataset...")

    # Path video
    normal_video_path = '/content/drive/MyDrive/sampel_video/orang_ramai_5.mp4'
    suspicious_video_path = '/content/drive/MyDrive/sampel_video/penculikan.mp4'

    # Validasi keberadaan file
    if not os.path.exists(normal_video_path):
        raise FileNotFoundError(f"Video normal tidak ditemukan di {normal_video_path}")
    if not os.path.exists(suspicious_video_path):
        raise FileNotFoundError(f"Video mencurigakan tidak ditemukan di {suspicious_video_path}")

    print("Memproses video normal...")
    normal_features = process_video(normal_video_path)
    if normal_features is None:
        raise ValueError("Gagal memproses video normal")

    print("Memproses video mencurigakan...")
    suspicious_features = process_video(suspicious_video_path)
    if suspicious_features is None:
        raise ValueError("Gagal memproses video mencurigakan")

    # Gabungkan fitur dan buat label
    X = np.vstack([normal_features, suspicious_features])
    y = np.hstack([
        np.zeros(len(normal_features)),
        np.ones(len(suspicious_features))
    ])

    print(f"Dataset siap: {len(X)} sampel total")
    return X, y

def train_classifier():
    """
    Melatih model klasifikasi.
    """
    try:
        print("Memulai pelatihan model...")
        X, y = prepare_dataset()

        # Bagi data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        # Normalisasi
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Latih model
        clf = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=42
        )
        clf.fit(X_train_scaled, y_train)

        # Evaluasi model
        train_score = clf.score(X_train_scaled, y_train)
        test_score = clf.score(X_test_scaled, y_test)
        print(f"Akurasi training: {train_score:.2%}")
        print(f"Akurasi testing: {test_score:.2%}")

        return clf, scaler

    except Exception as e:
        print(f"Error dalam pelatihan model: {str(e)}")
        return None, None

def analyze_video(video_path, clf, scaler):
    """
    Menganalisis video baru.
    """
    if clf is None or scaler is None:
        print("Error: Model klasifikasi atau scaler tidak tersedia")
        return None, None

    if not os.path.exists(video_path):
        print(f"Error: File video tidak ditemukan di {video_path}")
        return None, None

    try:
        features = process_video(video_path)
        if features is None:
            print("Error: Gagal mengekstrak fitur dari video")
            return None, None

        # Normalisasi fitur
        features_scaled = scaler.transform(features)

        # Prediksi
        predictions = clf.predict(features_scaled)
        suspicious_ratio = np.mean(predictions)

        return predictions, suspicious_ratio

    except Exception as e:
        print(f"Error dalam analisis video: {str(e)}")
        return None, None

# Jalankan proses training
print("Memulai proses training...")
clf, scaler = train_classifier()

if clf is not None and scaler is not None:
    # Analisis video
    test_video_path = '/content/drive/MyDrive/sampel_video/bullying.mp4'
    print(f"\nMenganalisis video: {test_video_path}")

    result = analyze_video(test_video_path, clf, scaler)

    if result is not None:
        predictions, suspicious_ratio = result
        print(f"\nHasil Analisis:")
        print(f"Proporsi frame mencurigakan: {suspicious_ratio:.2%}")
        if suspicious_ratio > 0.5:
            print("Kesimpulan: Video mengandung aktivitas mencurigakan")
        else:
            print("Kesimpulan: Video menunjukkan aktivitas normal")
    else:
        print("Gagal menganalisis video")
else:
    print("Gagal melatih model")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Memulai proses training...
Memulai pelatihan model...
Memulai persiapan dataset...
Memproses video normal...
Processed 100 frames
Processed 200 frames
Memproses video mencurigakan...
Processed 100 frames
Processed 200 frames
Processed 300 frames
Processed 400 frames
Dataset siap: 625 sampel total
Akurasi training: 100.00%
Akurasi testing: 96.80%

Menganalisis video: /content/drive/MyDrive/sampel_video/bullying.mp4
Processed 100 frames
Processed 200 frames
Processed 300 frames

Hasil Analisis:
Proporsi frame mencurigakan: 65.25%
Kesimpulan: Video mengandung aktivitas mencurigakan


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Gunakan path lengkap ke video
video_path = '/content/drive/MyDrive/sampel_video/bullying.mp4'
cap = cv2.VideoCapture(video_path)

# Periksa apakah video berhasil dibuka
if not cap.isOpened():
    print("Error: Video tidak dapat dibuka. Periksa kembali path file.")
    exit()

# Parameter Optical Flow
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Ambil frame pertama
ret, old_frame = cap.read()
if not ret:
    print("Error: Tidak dapat membaca frame pertama dari video.")
    cap.release()
    exit()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, maxCorners=100,
                             qualityLevel=0.3, minDistance=7, blockSize=7)
if p0 is None:
    print("Tidak ada fitur yang dapat dilacak pada frame pertama.")
    cap.release()
    exit()

# Mask untuk menggambar lintasan
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Gunakan GaussianBlur untuk mengurangi noise
    frame_gray = cv2.GaussianBlur(frame_gray, (5, 5), 0)

    # Hitung Optical Flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Validasi Optical Flow
    if p1 is None or st is None:
        print("Gagal menghitung optical flow pada frame ini.")
        continue

    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Deteksi ulang fitur jika terlalu sedikit
    if len(good_new) < 5:
        print("Fitur terlalu sedikit, mendeteksi ulang...")
        p0 = cv2.goodFeaturesToTrack(frame_gray, mask=None, maxCorners=100,
                                     qualityLevel=0.3, minDistance=7, blockSize=7)
        if p0 is None:
            print("Tidak ada fitur yang ditemukan.")
            break
        mask = np.zeros_like(frame)
        continue

    # Gambar lintasan pergerakan fitur
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 0, 255), -1)

    img = cv2.add(frame, mask)
    cv2_imshow(img)

    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

    # Break jika ada input tombol (biasanya pada local, di Colab tidak berfungsi sama)
    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
